In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

OUTPUT_DIR = '../output'
UPLOAD_DIR = '../output/uploads'

In [ ]:

# pd.set_option('max_columns', None)



df = pd.read_csv("../data/amazon-purchases-survey-v0_January+27,+2023_08.24.csv")
df.head(2)

In [8]:
# link file URLs to response IDs. 

(df
 [['ResponseID', 'Q43_Url', 'Q43_Name', 'download']][2:]
 # get only rows with files
 .assign(nofile = lambda x: pd.isna(x.Q43_Name))
 .query('nofile == False').drop(['download', 'nofile'], axis=1)
 # get response ID
 .assign(ResponseID = lambda x: x.Q43_Name.str.split('.').str[0])
 .rename({'Q43_Url': 'file_url', 'Q43_Name': 'file_name', 'ResponseID': 'response_id'}, axis=1)
).to_csv(os.path.join(OUTPUT_DIR, 'uploads.csv'))

---

# Downloading Data (obsolete)

This did not work / we couldn't download the files—qualtrics requires authentication before downloading from the links in the datasets, and setting that up in a script is a pain.

Instead, download from [here](https://mit.co1.qualtrics.com/results/?surveyId=SV_cMiItXdF95DdF5A&resultsDashboardOptIn=false#/surveys/SV_cMiItXdF95DdF5A/containers/62646e2371615f0011773859/pages/Page_57d1e889-db75-48f1-9587-9eb77ecb4ed7), it'll give you a zip file with all user uploads.

In [9]:
df = pd.read_csv(os.path.join(OUTPUT_DIR, 'uploads.csv'), index_col=0)

In [10]:
df.file_url.iloc[0]

'https://mit.co1.qualtrics.com/Q/File.php?F=F_bHQXnRUlnMc9qBw'

In [24]:
cookies = {
    'BIGipServer~nist~oc11-idp-core-prod-0': '204343570.0.0000',
    'JSESSIONID': '14C18DB6A38818A9125EE1C9F3778F1C',
    'shib_idp_session': '4ae3d2b7a98984bfb83364458a9de127c3f1c7d264804f3691a5f6bb704e19a2',
    'shib_idp_session_ss': 'AApzZWNyZXQyMjQxIlczsw8WR5eEB7PMwIy4EiTygjv1uSiLWfO1%2BF1q3ZqWhrRq81O5JOcUmKw4gbSz95EaIHXWK7BN9EUH3wcsSG6ep5RQo9Oc%2Fao9xcFk7OmVR3G%2FwCQy91zkZFvZhZ9VBntOHCZI8jXGtMjqelCIXsnItRh6UV%2FcwX8nyrBkLwVNCsfvd%2Fbu22BR0caKhp4fI34JdNMMaaGQj5pHa72geAwPKRNmVl4kNbJRROaj0DoVN1XVcgqVUPk8wO%2FzsM59bYZBIAlIWMQ39GoLyYIqBtMZ1%2BpptUBTxZkZ2SgoZCLDA5AnyMKlRgB3aJrcxow2c%2FMZPwx1xzGCzIpqfBTtYXeiCjEFdkjpaHXTrXEpI2y52E3TeqzSpKeGiVXal441kaR4jntvqZJcedIi3SPSqKbH0duP5%2FnPXMFI0h841JflghEOZZxO20%2B7Xbj0hWBzy67tu0pm'
}

In [25]:
def download_file(url, fname=None, extension='.csv', outpath=os.path.join(OUTPUT_DIR, 'uploads')):
    import requests
    # make output dir if it doesnt exist
    not os.path.exists(outpath) and os.makedirs(outpath)
    # construct filename from URL (works for our URL structures, not generally)
    filename = fname if fname != None else url.split('=')[1]
    filepath = os.path.join(outpath, f'{filename}{extension}')
    # HTTP request for file
    resp = requests.get(url, cookies=cookies)
    with open(filepath, 'wb') as f:
        f.write(resp.content)
    return filepath

In [20]:
from multiprocess import Pool
with Pool(16) as p:
    r = list(tqdm(p.imap(download_file, df.file_url)))


1772it [01:12, 24.39it/s]Process ForkPoolWorker-10:
Process ForkPoolWorker-18:
Process ForkPoolWorker-19:
Process ForkPoolWorker-14:
Process ForkPoolWorker-15:
Process ForkPoolWorker-7:
Process ForkPoolWorker-13:
Process ForkPoolWorker-12:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-8:



KeyboardInterrupt: 

Process ForkPoolWorker-9:
Process ForkPoolWorker-11:
Process ForkPoolWorker-17:
Process ForkPoolWorker-20:


# Exploring

In [ ]:
amazon_client_secret = '42e063754288f3cc6dcbdeeb7cf2c143f517e8c2944f40c3361e03a7c6aa5b5a'
amazon_client_id = 'amzn1.application-oa2-client.57b4cd00caf4442ca6ea6eeac23db8af'



In [28]:
df = pd.read_csv('../output/uploads/R_00CiHtfJgXWfETT~R_3ptgKiffuIHUepm.csv')

In [29]:
df

,Order Date,Purchase Price Per Unit,Quantity,Shipping Address State,Title,ASIN/ISBN (Product Code),Category,Survey ResponseID
0,11/22/18,$39.99,1,NaN,PlayStation Plus: 12 Month Membership [Digital...,B004RMK5QG,VIDEO_GAME_CONSOLE,R_3ptgKiffuIHUepm
1,11/18/19,$13.99,1,PA,Spigen Ultra Hybrid Designed for Apple iPhone ...,B07GTCC5X2,CELLULAR_PHONE_CASE,R_3ptgKiffuIHUepm
2,11/18/19,$9.00,1,PA,Amazon Basics Mini DisplayPort Male to HDMI Ma...,B0134V3KIA,ELECTRONIC_CABLE,R_3ptgKiffuIHUepm
3,11/18/19,$9.99,1,PA,NaN,B07F462SXM,NaN,R_3ptgKiffuIHUepm
4,12/26/19,$9.99,1,PA,OMOTON [2 Pack] Screen Protector Compatible wi...,B077XGL4PG,SCREEN_PROTECTOR,R_3ptgKiffuIHUepm
...,...,...,...,...,...,...,...,...
79,05/26/22,$9.99,1,PA,Goose Game meme metal enamel Pin,B084NXG8WM,APPAREL_PIN,R_3ptgKiffuIHUepm
80,05/26/22,$9.99,1,PA,Shocked Patrick - Spongebob Squarepants - 1.50...,B07CPLRZCS,APPAREL_PIN,R_3ptgKiffuIHUepm
81,05/26/22,$14.00,1,PA,OFFICIAL Star Wars Darth Vader Pin | Exclusive...,B075VPRVK2,APPAREL_PIN,R_3ptgKiffuIHUepm
82,06/29/22,$90.93,1,PA,AutoShack ASCA16545 Front Lower Control Arms w...,B08DJCSRRS,VEHICLE_CONTROL_ARM,R_3ptgKiffuIHUepm


# Unique products for scraping

In [28]:
import glob
import pandas as pd
from tqdm import tqdm 

files = glob.glob('../output/uploads/*.csv')
ASIN_COL = 'ASIN/ISBN (Product Code)'
product_file = '../output/all_products.txt'
product_set = set()

for f in tqdm(files):
    df = pd.read_csv(f)
    if ASIN_COL in df.columns:
        product_set.update(list(df[ASIN_COL].dropna()))
#    p = products + list(df[ASIN_COL])
#    products = list(set(p))
with open(product_file, 'w+') as f:
    asins = [f'{p}\n' for p in product_set]
    f.writelines(asins)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2767/2767 [00:05<00:00, 552.69it/s]


In [29]:
with open('../output/search_results_output.jsonl', 'r') as f:
    lines = f.readlines()
    

In [36]:
import json
asins = [json.loads(l)['ASIN'] for l in lines]

In [37]:
len(asins)

65

In [39]:
len(set(asins))

59

In [35]:
asins[0]['ASIN']

'B081RDDN1X'

In [40]:
with open('../output/all_products.txt', 'r') as f:
    all_products = f.readlines()

In [41]:
all_products[:10]

['B081RDDN1X\n',
 'B01N2QN4YP\n',
 '0060899190\n',
 'B019PC91MQ\n',
 'B001T8W4CO\n',
 'B0829QTSLT\n',
 'B08CLMTTQ4\n',
 '0983710511\n',
 'B075HQNLTD\n',
 'B07DL8NHX7\n']

In [42]:
len(set(all_products))

566975

In [43]:
all_products[:100]

['B081RDDN1X\n',
 'B01N2QN4YP\n',
 '0060899190\n',
 'B019PC91MQ\n',
 'B001T8W4CO\n',
 'B0829QTSLT\n',
 'B08CLMTTQ4\n',
 '0983710511\n',
 'B075HQNLTD\n',
 'B07DL8NHX7\n',
 'B07YL1PTMQ\n',
 'B093H7FTHB\n',
 'B07S7FSLF3\n',
 'B084T9MMH6\n',
 'B07NSW1R3T\n',
 'B01AXJFVBK\n',
 'B07N38MMTT\n',
 'B07K2J715H\n',
 'B07TTGZV35\n',
 'B07GBL679N\n',
 'B082HND82N\n',
 'B08XW2GKHS\n',
 'B00009R6VZ\n',
 'B08CR7WZVP\n',
 'B07GH7TJR9\n',
 'B09LVZ3T6C\n',
 'B08LDJHM4G\n',
 'B08FGNYVM5\n',
 'B07VF559F3\n',
 'B071VWCRBD\n',
 'B07SH66P35\n',
 'B005GLGY6C\n',
 'B08VR9HYC7\n',
 'B0823KXWJS\n',
 'B07PRSXLPC\n',
 'B09TLCRTKS\n',
 'B08FDKJQP4\n',
 'B09BBZ8F3D\n',
 'B082B221J1\n',
 'B07VB3PYKW\n',
 'B00020ICLC\n',
 '0898987024\n',
 'B00LJ1K4Q4\n',
 'B08LV588XQ\n',
 'B08X4L1HK3\n',
 'B002H9W9XI\n',
 'B07TQ7G3L3\n',
 'B07MMDC92C\n',
 '9332547017\n',
 'B07Q3XKCQ2\n',
 'B0751KBYDX\n',
 'B07538VLVX\n',
 'B0754ZWNBL\n',
 'B00DFC9ZCO\n',
 'B083HGG8PF\n',
 'B08RN7RYYK\n',
 'B07JQ1G3PJ\n',
 'B004LGX7XG\n',
 'B07H8BVWD1\n

In [25]:
products = list(product_set)

In [26]:
products[:5]

['B06X9V77XY', '149266247X', 'B07XQK68TM', 'B09STQCYW4', 'B09636FL56']

In [17]:
len(product_set)

567756

In [12]:
files

['../output/uploads/R_iC5Lst2FAei1oSF~R_Olel4p5FgLnzHVf.csv',
 '../output/uploads/R_aB3wdeNTA3mY9ku~R_1GBQ45wkJVX0xCi.csv',
 '../output/uploads/R_Hfq3w6RLDvL5w4Y~R_3oI6mZDt47OaXO5.csv',
 '../output/uploads/R_tlZiP7Bti5vxGys~R_1qU5mT6xlj8KYSL.csv',
 '../output/uploads/R_zbSUiJPaWZ9YYmd~R_To1AwJ2KphjS9Il.csv',
 '../output/uploads/R_RWcznzAv89vSFaM~R_1mf98AYyqqPtPn7.csv',
 '../output/uploads/R_7ikGPicNjiJaLpz~R_cMWCYGFYS7cUeM9.csv',
 '../output/uploads/R_tUwrzhJnBteMhbD~R_1PYPmKqQr85ZxYc.csv',
 '../output/uploads/R_56Y74FYzXVW7g6X~R_2s0jSMfxg63zSw5.csv',
 '../output/uploads/R_I2GvJXdIaBMgvRt~R_1k1zPs6FKsqm1Ib.csv',
 '../output/uploads/R_XJn9WBYSRkji0NU~R_25RsmPLCHnSkLaN.csv',
 '../output/uploads/R_uNpXNDp89KPLz2Q~R_rrtchX36usn87xT.csv',
 '../output/uploads/R_Kx9lo2waEnlMSIg~R_32Wsk7ziaut6JrW.csv',
 '../output/uploads/R_DrGWXWsnQZP1GEw~R_by1wIaDyysxuQY9.csv',
 '../output/uploads/R_e1zr0EEEX6sxLoS~R_OQeQHVDhrK9ane1.csv',
 '../output/uploads/R_7R966wCqfXmnQgq~R_0jFt3K3i4558lbj.csv',
 '../out

In [10]:
!ls ../output/uploads/*.csv

../output/uploads/R_00CiHtfJgXWfETT~R_3ptgKiffuIHUepm.csv
../output/uploads/R_04C8mk2TpfpKp9a~R_31gUuqxSkTAfMPg.csv
../output/uploads/R_05Edp9cwJVh0OaN~R_2bO84Ktdkq0012R.csv
../output/uploads/R_0639xkQ0tMzocSZ~R_30f0gTKJr78YCy4.csv
../output/uploads/R_07mL6CdUTGJxKjx~R_1GT7ULa9rUiWPzn.csv
../output/uploads/R_0DJmvfxPxHvWPul~R_2w5m3XJMrF4EdY0.csv
../output/uploads/R_0GppBhQ7aOKsRJe~R_2ZTFfLAcuwm22Yh.csv
../output/uploads/R_0Ihpcn7rbzz5aGp~R_1JPaQpfztybIYnz.csv
../output/uploads/R_0J0IQYozIGWD9kF~R_21EtjPQuJMuZSHZ.csv
../output/uploads/R_0JfZg2IdXMXyr3N~R_2S9b47bd0M7IKFu.csv
../output/uploads/R_0KWtZEydS00n5z3~R_2PczkBPRGmqC8Ru.csv
../output/uploads/R_0Koun4KPYGcwAnu~R_1lhEybvgNui8FOA.csv
../output/uploads/R_0MQSStNKwUTvHQo~R_1ewXi7AGxIOjtC1.csv
../output/uploads/R_0P1LKfBZPjt4l4z~R_33lsJEmPv7wPank.csv
../output/uploads/R_0PKGje4qH1ztDDz~R_1JL4SqKTl0o7xEN.csv
../output/uploads/R_0R2rbNLqAazq77x~R_23LGwgsgRxQiNJ7.csv
../output/uploads/R_0TLwa6QCbJI5hdJ~R_2Pe6tHUCkAWyCiD.csv
../output/uplo

../output/uploads/R_sOEQoIcU4Qvot7w~R_2AQiu69aAVY6utf.csv
../output/uploads/R_sRcpwKr7PWbMW3b~R_8igkqu4jEaogJzj.csv
../output/uploads/R_sRvLz2hQf7MCo1L~R_pcLKzqCXX1JsGUV.csv
../output/uploads/R_sUO0I3OTNiJfbMn~R_xFpPWhOPnXmXPlT.csv
../output/uploads/R_sURCF7F19BhVynC~R_2PqhFgM0oAFa4ho.csv
../output/uploads/R_sWzkcvhJw6SjfYM~R_9WSxduvirxNr42t.csv
../output/uploads/R_sX9IIz365KPiT4p~R_2BxKzKp6GeWepzD.csv
../output/uploads/R_saqO4u1qq84cJxD~R_Ybh09t36EFaS3jb.csv
../output/uploads/R_sbAUrSgdS0BuWTM~R_3k5JabbpW5kTO7M.csv
../output/uploads/R_sc63ynNQxqjGq8j~R_1K9yOc1WbKzImgH.csv
../output/uploads/R_shF9CjLqBqGYVzY~R_3wT1t9duSUIPC5b.csv
../output/uploads/R_shZdVQXnDFbBlfP~R_8ro09iHf6361XLX.csv
../output/uploads/R_siQ42bVibYuEIja~R_XTmxVoQGo9qhbyx.csv
../output/uploads/R_sj4nLJUJeaBhwDP~R_1pDUHsNfwFbm6xw.csv
../output/uploads/R_sjzzxkdkODOVhry~R_exnLc1H6oypi9vr.csv
../output/uploads/R_skp5ELIwb4Lg9yK~R_31MddZPuw2VomsN.csv
../output/uploads/R_slbVIIFM0l42QYH~R_23dl1M82ZW221h1.csv
../output/uplo